In [ ]:
from pathlib import Path

import altair as alt
import polars as pl

In [ ]:
data_dir = Path(r"Q:\CMP\LOS Monitoring 2023\transit")
figs_dir = Path(
    r"Q:\CMP\reports\CMPSF 2023\Draft\figures\multimodal_performance\transit"
)
color_ampm_domain = ["a.m.", "p.m."]
color_ampm_range = ["#8cb7c9", "#d3d655"]
speed_filename_stem = "transit-speed-avg-historical"
variability_filename_stem = (
    "transit-speed-avg_coefficient_of_variation-historical"
)
auto_transit_speed_ratio_filename_stem = (
    "auto_transit_speed_ratio-avg-historical"
)

In [ ]:
def melt(df_wide, metric, citywide_vs_downtowncore):
    id_vars = ["year"]
    if citywide_vs_downtowncore:
        id_vars.append("geography")
    df_long = (
        df_wide.melt(
            id_vars=id_vars,
            value_vars=["a.m.", "p.m."],
            variable_name="peak period",
            value_name=metric,
        )
        # need to convert year to date(time) for plotting the year axis
        .with_columns(pl.date(pl.col("year"), 1, 1).alias("date"))
    )
    return df_long


def load_csv(filepath, metric, citywide_vs_downtowncore):
    rename_cols = {"AM": "a.m.", "PM": "p.m."}
    df_wide = pl.read_csv(filepath)
    if citywide_vs_downtowncore:
        dfs_wide = [
            df_wide.select(
                "year",
                pl.col("citywide-AM").alias("AM"),
                pl.col("citywide-PM").alias("PM"),
                pl.lit("citywide").alias("geography"),
            ).rename(rename_cols),
            df_wide.select(
                "year",
                pl.col("downtown_core-AM").alias("AM"),
                pl.col("downtown_core-PM").alias("PM"),
                pl.lit("downtown core").alias("geography"),
            ).rename(rename_cols),
        ]
        dfs_long = [
            melt(df_wide, metric, citywide_vs_downtowncore)
            for df_wide in dfs_wide
        ]
        df_long = pl.concat(dfs_long)
    else:
        df_wide = df_wide.rename(rename_cols)
        df_long = melt(df_wide, metric, citywide_vs_downtowncore)
    return df_long.to_pandas()


def plot(
    df_long,
    save_filepath,
    column_name,
    y_title,
    citywide_vs_downtowncore,
):
    y = alt.Y(f"{column_name}:Q").title(y_title)
    if y_title == "transit speed coefficient of variation":
        y = y.axis(format="%")
    chart = alt.Chart(df_long).mark_line()
    if citywide_vs_downtowncore:
        chart = chart.encode(
            alt.X("date:T"),
            y,
            color=alt.Color("peak period:N").scale(
                domain=color_ampm_domain, range=color_ampm_range
            ),
            strokeDash=alt.StrokeDash("geography:N"),
            tooltip=[
                "year:Q",
                "geography:N",
                "peak period:N",
                f"{column_name}:Q",
            ],
        )
    else:
        chart = chart.encode(
            alt.X("date:T"),
            y,
            color=alt.Color("peak period:N").scale(
                domain=color_ampm_domain, range=color_ampm_range
            ),
            tooltip=[
                "year:Q",
                "peak period:N",
                f"{column_name}:Q",
            ],
        )
    chart.save(save_filepath, scale_factor=2)
    return chart  # .interactive()


def historic_trend_fig(
    data_dir, figs_dir, filename_stem, y_title, citywide_vs_downtowncore=False
):
    data_filepath = Path(data_dir) / f"{filename_stem}.csv"
    fig_filepath = Path(figs_dir) / f"{filename_stem}.png"
    column_name = "metric"  # placeholder for the functions below
    df_long = load_csv(data_filepath, column_name, citywide_vs_downtowncore)
    return plot(
        df_long, fig_filepath, column_name, y_title, citywide_vs_downtowncore
    )

In [ ]:
historic_trend_fig(
    data_dir,
    figs_dir,
    speed_filename_stem,
    "transit speed (mph)",
    citywide_vs_downtowncore=True
)

In [ ]:
historic_trend_fig(
    data_dir, figs_dir, variability_filename_stem, "transit speed coefficient of variation"
)

In [ ]:
historic_trend_fig(
    data_dir, figs_dir, auto_transit_speed_ratio_filename_stem, "auto / transit speed ratio"
)